<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/main/recommendqtion-engine-udemy-final-solution-v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [2]:
# importing libraries
# src https://www.youtube.com/watch?v=ueKXSupHz6Q

In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from numpy import save
from numpy import asarray
from numpy import load

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# show available language
print(stopwords.fileids())

['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [5]:
# Load the data 
from google.colab import files
uploaded = files.upload()

In [6]:
#Store the data 
df = pd.read_csv('udemy-data-v2.csv')
# show the first 3 rows of data 
df['title'].head(3)

0                        NoSQL Databases For Beginners
1                            Tai Chi Foundation Course
2    Music Pedagogy: 7 advices to teach a musical i...
Name: title, dtype: object

In [7]:
df.shape


(3695, 13)

In [8]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [9]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple
1,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist
2,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De..."


In [10]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [11]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [12]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(10)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,num_subscribers,avg_rating,num_reviews,created,important_features
0,1117836,NoSQL Databases For Beginners,<p>NoSQL (Non-SQL or Not-only-SQL) databases a...,Learn to Store and Retrieve Data from a Datab...,Development,Database Design & Development,Bluelime Learning Solutions,Learning made simple,€94.99,1178,3.500000,128,2017-02-17T12:46:11Z,NoSQL Databases For Beginners <p>NoSQL (Non-SQ...
1,1117864,Tai Chi Foundation Course,<p>Whether you're a beginner or experienced st...,Traditional methods. Modern teaching.,Health & Fitness,General Health,Aamir Rafi,Tai Chi practitioner & life long martial artist,€89.99,5,0.000000,0,2017-02-17T13:28:15Z,Tai Chi Foundation Course <p>Whether you're a ...
2,1117988,Music Pedagogy: 7 advices to teach a musical i...,<p><strong>Are you a music teacher?</strong></...,Learn how to organize and improve your classes.,Music,Music Fundamentals,Ilse Lozoya,"University Professor, Piano Teacher, Course De...",€39.99,1345,4.000000,7,2017-02-17T15:50:16Z,Music Pedagogy: 7 advices to teach a musical i...
3,1118012,Ultimate Power Electronics for Electrical Powe...,"<p><strong>""Ultimate Power Electronics Course ...",Learn everything about power electronics for e...,Teaching & Academics,Engineering,Ahmed Mahdy / Khadija Academy,Electrical Power Engineer,€94.99,3186,4.592593,481,2017-02-17T16:15:02Z,Ultimate Power Electronics for Electrical Powe...
4,1118046,Facebook Advertising: Create Converting Facebo...,<p><strong>Over 8000 students have joined this...,"Learn how to select your niche, identify marke...",Marketing,Social Media Marketing,Barry North,"Award Winning ""Sexiest Voice""",€89.99,10498,4.200000,68,2017-02-17T16:48:28Z,Facebook Advertising: Create Converting Facebo...
5,1118072,Planning & Scheduling: Be the Professional fro...,<p><strong>Planning and Scheduling</strong> is...,Project Planning and Scheduling Management cou...,Business,Project Management,"Construction Project Management Guru (PSP, PMI...",Promoting Professionalism in Planning and Sche...,€109.99,3393,4.074074,615,2017-02-17T17:08:03Z,Planning & Scheduling: Be the Professional fro...
6,1118146,Wie du einen genialen Vortrag hältst,<p>Dein Vortrag ist deine Chance</p> <p>Mit d...,Mit Simon (Profi-Speaker) und Michi (NLP-Train...,Personal Development,Career Development,Simon Schnetzer & Michael Schmidt,Speaker & Persönlichkeitsentwickler,€44.99,345,4.100000,31,2017-02-17T17:50:36Z,Wie du einen genialen Vortrag hältst <p>Dein V...
7,1118268,Top Five Leadership Behaviors,<p>Every Leader Needs To Gain Their Team's Tru...,Strong Leaders Don't Just Happen: Learn These ...,Personal Development,Leadership,Donna Galatas,HR Vice Empress / CEO of The Galatas Group,€94.99,129,4.350000,27,2017-02-17T19:38:01Z,Top Five Leadership Behaviors <p>Every Leader ...
8,1118274,The Visual Guide on How Neural Networks Learn ...,<p>Course Achievements (January 2021):&nbsp;</...,The BEST Resource for Understanding Neural Net...,Development,Data Science,Mauricio Maroto,"+10,000 students and growing!",€94.99,3163,4.700000,143,2017-02-17T19:54:28Z,The Visual Guide on How Neural Networks Learn ...
9,1118300,Aprende a crear Plugins para WordPress desde c...,<p><strong>Más de 60 reseñas fantásticas !!! 4...,Es hora de saltar al siguiente paso en WordPre...,Development,Web Development,Gilbert Rodríguez,Desarrollador Web,€94.99,1457,4.500000,243,2017-02-17T20:32:46Z,Aprende a crear Plugins para WordPress desde c...


In [13]:
# creating stop word
stop_words =  stopwords.words('arabic') + stopwords.words('azerbaijani') + stopwords.words('danish') + stopwords.words('dutch') + stopwords.words('english') + stopwords.words('finnish') + stopwords.words('french') + stopwords.words('german') + stopwords.words('greek') + stopwords.words('hungarian') + stopwords.words('indonesian') + stopwords.words('italian') + stopwords.words('kazakh') + stopwords.words('nepali') + stopwords.words('norwegian') + stopwords.words('portuguese') + stopwords.words('romanian') + stopwords.words('russian') + stopwords.words('slovene') + stopwords.words('spanish') + stopwords.words('swedish') + stopwords.words('tajik') + stopwords.words('turkish')
print(stop_words)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [34]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    NoSQL Databases For Beginners NoSQL (Non-SQL N...
1    Tai Chi Foundation Course Whether beginner exp...
2    Music Pedagogy: 7 advices teach musical instru...
Name: important_features, dtype: object
[[1.         0.08476291 0.14103213 ... 0.1857592  0.0392318  0.01588277]
 [0.08476291 1.         0.17646815 ... 0.04960893 0.03322965 0.        ]
 [0.14103213 0.17646815 1.         ... 0.07030453 0.02638789 0.        ]
 ...
 [0.1857592  0.04960893 0.07030453 ... 1.         0.02567836 0.01126205]
 [0.0392318  0.03322965 0.02638789 ... 0.02567836 1.         0.0326893 ]
 [0.01588277 0.         0.         ... 0.01126205 0.0326893  1.        ]]


In [35]:
# save data in file 
data = asarray(cs)
save('data.npy', data)


In [45]:
# Loading data from a file
cs = load('data.npy')

In [52]:
# Get the shape of the cosine similarity matrix 
cs.shape


(3695, 3695)

In [53]:
# Get the title of the course that the user likes 
title_to_search = 'R: Complete Data Visualization Solutions'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251

1120


In [54]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [55]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [56]:
#Print the sorted scores
print(sorted_scores)

[(1288, 0.6517829693461817), (3352, 0.6336417944983029), (311, 0.6143874701401542), (348, 0.6053071551610354), (1385, 0.6006228229095734), (3389, 0.5864772011419507), (1846, 0.5828881539452699), (3021, 0.5808306544047922), (2593, 0.5764270850195026), (1218, 0.5731591185889906), (1254, 0.562435364893313), (3183, 0.5550746820173065), (2925, 0.5521413110542073), (2903, 0.5477451626096642), (2709, 0.5402865759274865), (1800, 0.5299177916343117), (146, 0.5258954352884402), (1176, 0.5255856621426475), (2277, 0.5250950121397029), (900, 0.5167993565700254), (2753, 0.49986113319467246), (2690, 0.49830662723435776), (2489, 0.49466732251740325), (2761, 0.491797736212598), (1150, 0.4850297251984688), (1383, 0.48055210575830914), (3380, 0.4770402718922698), (1189, 0.47436370407191397), (1493, 0.47370496391502936), (3020, 0.4708635887436753), (1280, 0.46643997698990636), (3448, 0.46523653055090736), (0, 0.46463829054797623), (96, 0.46345380986054185), (700, 0.4539808207757947), (2447, 0.449099939196

In [57]:
# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended courses : ', title_to_search, '\nare:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  #course_headline = df[df.index == item[0]]['headline'].values[0]
  num_subscribers = df[df.index == item[0]]['num_subscribers'].values[0]
  print(course_id, course_title, " | " + str(num_subscribers) )
  j=j+1
  if j>100:
    break

The 7 most recommended courses :  R: Complete Data Visualization Solutions 
are:

ID  Course Title
1166654 Real-World Data Science with Spark 2  | 335
1233522 R: Programming and Data Science  | 40
1129804 Haskell: Data Analysis Made Easy  | 604
1131092 R: Complete Machine Learning Solutions  | 564
1170002 Learning Path: R: Real-World Data Mining With R  | 127
1234834 D3 4.x: Mastering Data Visualization  | 368
1184730 Learning Path: Python: Effective Data Analysis Using Python  | 396
1222300 SGLearn@Python for Data Science & Machine Learning Bootcamp  | 20
1208478 Hadoop and Big Data for Absolute Beginners  | 319
1164440 Learning Path: Julia: Explore Data Science with Julia  | 703
1165532 Effective Data Visualization  | 734
1227924 Implementing a Data Warehouse with Microsoft SQL Server  | 2167
1219062 Julia: Performing Statistical Computations  | 47
1218132 How To Create Dashboards With Google Data Studio  | 190
1212178 Learning Path: From Python Programming to Data Science  | 378
118